In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time, re, json
import pandas as pd

In [2]:
def scrapNormal():
    finalScrap = [] # Creates the list for storing dictionaries

    for i in range(10):
        temp = {}
        
        keyList = ['company_name', 'company_link', 'company_logo', 'company_address', 'country', 
           'company_phones', 'company_fax', 'company_emails', 'company_website', 'company_contacts', 'company_industry']
        
        for j in keyList: # Sets all default value for every key
            temp[j] = '-'
        
        temp['company_address'] = [{'address_1': '-'}]
        temp['company_phones'] = []
        temp['company_fax'] = []
        temp['company_industry'] = ['-']
        
        time.sleep(1)
        
        try: # Scrap Company logo
            companyLogo = driver.find_element_by_xpath("/html/body/div[2]/form/div[3]/table[2]/tbody/tr/td[2]/div/div/div[3]/div/div/div/table/tbody/tr["+str(i+2)+"]/td/table/tbody/tr[1]/td[1]/a/img").get_attribute("src")
        except NoSuchElementException:
            companyLogo = ''
        
        # Opens Every Company Link in Page
        driver.find_element_by_xpath("//*[@id='ContentPlaceHolder1_dgrdCompany_Hyperlink2_"+str(i)+"']").click()
        time.sleep(1)
        
        try: # Tries to scrap the text inside the page
            rawText = driver.find_element_by_xpath("/html/body/div[2]/form/div[3]/table[2]/tbody/tr/td[2]/div/div/div[1]/table/tbody/tr/td/table/tbody/tr/td[2]").text
        except:
            finalScrap = differentScrap() # If the scrap goes here, it means the web page structure is different
            return finalScrap
            
        rawData = list(filter(None, rawText.split('\n'))) # Splits the <br> from html into list
        
        # Sets the default temporary value before inserting to list finalResult
        companyTelephone = []
        companyFax = []
        companyEmail = '-'
        companyWebsite = '-'
        companyContact = ''
        contactIndex = 0

        for i in rawData: # Scraps the data before Company Contacts and after Address
            if i == 'Contact':
                break
            elif 'Fax:' in i:
                companyFax = i
            elif 'Tel: ' in i:
                companyTelephone = i
            elif 'Email: ' in i:
                companyEmail = i
            elif 'Website: ' in i:
                companyWebsite= i

            contactIndex += 1

        
        # This variable is built in order to check if company contacts has more than one person
        nameState, emailState, jobState, telephoneState, helplineState = False, False, False, False, False

        z = [] # final list of company contacts
        y = [] # the temporary values of company contacts
        w = [] # the temporary keys of company contacts

        count = contactIndex+1 # Gets the previous index from before when scraping before contacts

        for i in rawData[count:-1]: # checking if there is any comma in string, for example 'Mr Budhi, Director'
            if ',' in i:
                i = i.split(', ')
                rawData[count] = i[0]
                a = 0
                for j in i[1:]:
                    rawData.insert(count+a, j)
                    a+=1

                count+=1
            elif 'Advertisement' in i:
                rawData.pop(count)

            count +=1

        def checkState(state, contactList, listKey): # function to make sure if theres one or more person in company contact
            if state:
                count = 0
                for i in contactList:
                    if ':' in i:
                        i = i.split(': ') # Deleting ':'
                        contactList[count] = ''.join(i[1:])

                    count+=1

                contactList = list(filter(None, contactList)) # Deleting None and empty values

                contactData = {} # Initializing all fields just in case the contacts doesnt have all the values
                contactData['contact_name'] = '-'
                contactData['email'] = '-'
                contactData['phone'] = []
                contactData['helpline'] = '-'
                contactData['job_title'] = '-'

                for i in range(len(listKey)):
                    if listKey[i] == 'phone' or listKey[i] == 'helpline':
                        IntVar = int("".join(filter(str.isdigit, contactList[i])))# Changing Phone number to Integer
                        contactData[listKey[i]] = [IntVar]
                    else:
                        contactData[listKey[i]] = contactList[i] # Replacing the initialized value

                z.append(contactData)
                return [],[], True # Emptying the temporary keys and values list
            else:
                return contactList, listKey, False # This line means if the values are still related to the same contact person

        for i in rawData[contactIndex+1:-1]: 
            if (('Mr' in i) or ('Ms' in i)): # Scraping contact name
                y,w,r = checkState(nameState, y, w)
                
                if r:
                    nameState, emailState, jobState, telephoneState, helplineState = False, False, False, False, False
                
                nameState = True
                y.append(i)
                w.append('contact_name')
            elif ('@' in i): # Scraping contact emails
                y,w,r = checkState(emailState, y, w)
                if r:
                    nameState, emailState, jobState, telephoneState, helplineState = False, False, False, False, False
                    
                emailState = True
                y.append(i)
                w.append('email')
            elif ('Handphone:' in i): # Scraping contact phones
                y,w,r = checkState(telephoneState, y, w)
                
                if r:
                    nameState, emailState, jobState, telephoneState, helplineState = False, False, False, False, False
                    
                telephoneState = True
                y.append(i)
                w.append('phone')
            elif ('Helpline:' in i): # Scraping contact helplines
                y,w,r = checkState(helplineState, y, w)
                
                if r:
                    nameState, emailState, jobState, telephoneState, helplineState = False, False, False, False, False
                    
                helplineState = True
                y.append(i)
                w.append('helpline')
            elif ((('Mr' in i) or ('Ms' in i)) == False): # Scraping job title
                y,w,r = checkState(jobState, y, w)
                
                if r:
                    nameState, emailState, jobState, telephoneState, helplineState = False, False, False, False, False
                
                jobState = True
                y.append(i)
                w.append('job_title')

        y,w,r = checkState(nameState, y, w)

        count = 0
        for i in rawData:
            if ':' in i:
                i = i.split(': ') # Deleting all the :
                rawData[count] = ''.join(i[1:])

            count+=1

        rawData = list(filter(None, rawData)) # Making sure the data is clean before getting address and industries

        companyName = rawData[0]
        companyAddress = []
        companyCountry = '-'
        count = 2

        for i in rawData[2:]: # Scraping the address 
            x = bool(re.search('[a-zA-Z]', i))

            if x and ('(' in i) == False:
                companyAddress.append(i)
            else:
                companyCountry = companyAddress[count-3] # Scraping the country
                companyAddress.pop(count-3)
                break

            count += 1

        companyUrl = driver.current_url # Scraping the company link
        companyAddress = [{'address_1' : ' '.join(companyAddress)}]
        companyCategories = []
        count = 2

        while(True): # Scraping the company industry
            try:
                x = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_dtlCat"]/tbody/tr['+str(count)+']/td/a').text
                x = x.split('-> ')
                companyCategories.append(x[1])
            except NoSuchElementException:
                break

            count+=1

        temp['company_name'] = companyName
        temp['company_link']  = companyUrl
        temp['company_logo']  = companyLogo
        temp['company_address']  = companyAddress
        temp['country'] = companyCountry
        temp['company_phones'] = companyTelephone
        temp['company_fax'] = companyFax
        temp['company_emails'] = companyEmail
        temp['company_website'] = companyWebsite
        temp['company_contacts'] = z
        temp['company_industry'] = companyCategories

        for i in temp: # Cleanse the data for the last time
            if ':' in temp[i] and ('http' in temp[i]) == False:
                x = temp[i].split(': ')
                temp[i] = ''.join(x[1:])

            if ',' in temp[i] and ('address' in i) == False:
                x = temp[i].split(', ')
                temp[i] = x
        
        if isinstance(temp['company_phones'], str): # Turns phones into integer if the value is list
            IntVar = int("".join(filter(str.isdigit, temp['company_phones'])))
            temp['company_phones'] = [IntVar]
        else:
            tempVar = []
            for i in temp['company_phones']: # Turns phones into integer if the value is String
                IntVar = int("".join(filter(str.isdigit, i)))
                tempVar.append(IntVar)
            
            temp['company_phones'] = tempVar
            
        if isinstance(temp['company_fax'], str): # Turns fax into integer if the value is String
            IntVar = int("".join(filter(str.isdigit,temp['company_fax'])))
            temp['company_fax'] = [IntVar]
        else:
            tempVar = []
            for i in temp['company_fax']: # Turns fax into integer if the value is String
                IntVar = int("".join(filter(str.isdigit, i)))
                tempVar.append(IntVar)
            
            temp['company_fax'] = tempVar
            
        contactData = {} # Initialize Contact Data if the web has no Contact Field at all
        contactData['contact_name'] = '-'
        contactData['email'] = '-'
        contactData['phone'] = []
        contactData['helpline'] = '-'
        contactData['job_title'] = '-'
        
        if temp['company_contacts'] == []:
            temp['company_contacts'] = [contactData] # Turning the empty list to the initialized contact data
            
        finalScrap.append(temp) # add all the values to the final list
        driver.back() # go back to company link pages
    
    return finalScrap

In [3]:
def differentScrap():
    finalScrap = [] # Creates the list for storing dictionaries
    
    for i in range(10):
        time.sleep(1)
        if i > 0:
            driver.find_element_by_xpath("//*[@id='ContentPlaceHolder1_dgrdCompany_Hyperlink2_"+str(i)+"']").click()
        
        count = 1
        rawData = {}
        keyList = ['company_name', 'company_link', 'company_logo', 'company_address', 'country', 
           'company_phones', 'company_fax', 'company_emails', 'company_website', 'company_contacts', 'company_industry']
        
        for i in keyList: # Sets all default value for every key
            rawData[i] = '-'
            
        # Sets all default value for every key
        rawData['company_address'] = [{'address_1': '-'}]
        rawData['company_phones'] = []
        rawData['company_fax'] = []
        rawData['company_industry'] = ['-']
        
        tempDict = {}
        tempDict['contact_name'] = '-'
        tempDict['job_title'] = '-'
        tempDict['phone'] = []
        tempDict['email'] = '-'
        
        rawData['company_contacts'] = [tempDict]

        while(True):
            try:
                
                # tempKey is for the left-column side, while tempValue is the right-column side
                
                tempKey = driver.find_element_by_xpath('//*[@id="article_box3"]/table[2]/tbody/tr/td/div/table/tbody/tr/td/table/tbody/tr['+str(count)+']/td[1]').text
                tempValue = driver.find_element_by_xpath('//*[@id="article_box3"]/table[2]/tbody/tr/td/div/table/tbody/tr/td/table/tbody/tr['+str(count)+']/td[2]/div').text

                if 'Nature' in tempKey: # The Busines Nature has an additional empty row, so has to add count +1
                    count+=1
                elif 'Company Name' in tempKey: # Scraps company Name, Link and Logo
                    rawData['company_name'] = tempValue
                    rawData['company_link']  = driver.current_url
                    rawData['company_logo']  = '-'

                elif 'Other Address' in tempKey: # Scraps if there are Other Address Column
                    tempValue = tempValue.split('\n')
                    x = rawData['company_address']
                    y = {}
                    y['address_1'] = x
                    y['address_2'] = ', '.join(tempValue)
                    rawData['company_address']  = [y]
                elif 'Address' in tempKey: # Scraps Address column
                    tempValue = tempValue.split('\n')
                    if ' ' in tempValue[-1]: # split if the last element is not country
                        x = tempValue[-1].split(' ')
                        rawData['company_address']  = ' '.join(tempValue[:-1]) + ' '.join(x[:-1])
                        rawData['country'] = x[-1]
                    else: # if last element is country
                        rawData['company_address']  = ' '.join(tempValue[:-1])
                        rawData['country'] = tempValue[-1]
                elif 'Telephone' in tempKey: # Scraps company phone
                    if ',' in tempValue: # If has more than 1 company phone
                        x = tempValue.split(', ')
                        y = []
                        for i in x:
                            IntVar = int("".join(filter(str.isdigit, i)))
                            y.append(IntVar)
                        rawData['company_phones'] = y
                    else: # If has only 1 company phone
                        IntVar = int("".join(filter(str.isdigit, tempValue)))
                        rawData['company_phones'] = [IntVar]
                elif 'Fax' in tempKey: # Scraps company Fax
                    if ',' in tempValue: # If has more than 1 company fax
                        x = tempValue.split(', ')
                        y = []
                        for i in x:
                            IntVar = int("".join(filter(str.isdigit, i)))
                            y.append(IntVar)
                            
                        rawData['company_fax'] = y
                    else:  # If has only 1 company fax
                        IntVar = int("".join(filter(str.isdigit, tempValue)))
                        rawData['company_fax'] = [IntVar]
                elif 'Email' in tempKey: # gets email
                    rawData['company_emails'] = tempValue
                elif 'Website' in tempKey: # gets website
                    rawData['company_website'] = tempValue
                elif 'Contact' in tempKey:
                    tempDict = {} # Initialize if contact column doesnt have all the value
                    tempDict['contact_name'] = '-'
                    tempDict['job_title'] = '-'
                    tempDict['phone'] = []
                    tempDict['email'] = '-'
                    if ',' in tempValue :
                        tempValue = tempValue.split(', ')
                        for i in tempValue:
                            if '' in tempValue:
                                dummy = ''
                            elif '@' in i:
                                tempDict['email'] = i
                            elif (('Mr' in i) or ('Ms' in i)):
                                tempDict['contact_name'] = i
                            elif bool(re.search('[a-zA-Z]', i))==False:
                                IntVar = int("".join(filter(str.isdigit, i)))
                                tempDict['phone'] = [IntVar]
                            else:
                                tempDict['job_title'] = i
                        
                        rawData['company_contacts'] = [tempDict]
                    else: # Has only 1 value
                        if '' in tempValue:
                            dummy = ''
                        elif '@' in tempValue:
                            tempDict['email'] = tempValue
                        elif (('Mr' in tempValue) or ('Ms' in tempValue)):
                            tempDict['contact_name'] = tempValue
                        elif bool(re.search('[a-zA-Z]', tempValue))==False:
                            IntVar = int("".join(filter(str.isdigit, tempValue)))
                            tempDict['phone'] = [IntVar]
                        else:
                            tempDict['job_title'] = tempValue
                        
                        rawData['company_contacts'] = [tempDict]
                elif 'Categories' in tempKey:
                    if '\n' in tempValue : # Has more than 1 value
                        tempValue = tempValue.split('\n')
                        x = []
                        for i in tempValue:
                            i = i.split('-> ')
                            x.append(i[1])

                        rawData['company_industry'] = x
                    else: # If has only 1 value
                        tempValue = tempValue.split('-> ')
                        rawData['company_industry'] = [tempValue[1]]

                count += 2

            except NoSuchElementException:
                break
            
        for i in rawData:
            if ',' in rawData[i] and ('address' in i) == False: # Last Split
                x = rawData[i].split(', ')
                rawData[i] = x
                
        if isinstance(rawData['company_address'], str): # turns address into dict in list
            rawData['company_address'] = [{'address_1': rawData['company_address']}]
            

        finalScrap.append(rawData) # Adds all value to final list
        driver.back() # Back to company list page

    return finalScrap

In [4]:
def bonusPoint(finalResult): 
    columns = ['company_uid', 'company_name', 'country', 'company_website', 'company_address', 
           'company_phones', 'company_faxes', 'company_industry']
    data = []
    for i in range(len(finalResult)): # Creates the first sheet from finalResult list
        data.append([finalResult[i]['company_link'], 
                      finalResult[i]['company_name'],
                       finalResult[i]['country'], 
                       finalResult[i]['company_website'],
                      finalResult[i]['company_address'],
                      finalResult[i]['company_phones'],
                      finalResult[i]['company_fax'],
                      finalResult[i]['company_industry']])

    df_company_parsed = pd.DataFrame(data, columns = columns) 
    
    columns = ['company_uid', 'company_name', 'country', 'company_website', 'contact_name', 
           'contact_job_title', 'contact_phone']
    data = []
    for i in range(len(finalResult)):
        for j in finalResult[i]['company_contacts']: # Creates the second sheet from finalResult list
            tempList = []
            tempList.append(finalResult[i]['company_link'])
            tempList.append(finalResult[i]['company_name'])
            tempList.append(finalResult[i]['country'])
            tempList.append(finalResult[i]['company_website'])
            tempList.append(j['contact_name'])
            tempList.append(j['job_title'])
            tempList.append(j['phone'])
            data.append(tempList)

    df_contact_parsed = pd.DataFrame(data, columns = columns) 
    print('Exporting to Excel.. Please wait.') 
    with pd.ExcelWriter(r'bonusPoint.xlsx') as Aufar:  # Creates the excel file with two sheets
        df_company_parsed.to_excel(Aufar, sheet_name='Companies Parsed')
        df_contact_parsed.to_excel(Aufar, sheet_name='Contacts Parsed')

    print('Excel File Created. (bonusPoint.xlsx)')

In [5]:
def questionOne(finalResult):
    
    questionOne = []
    
    for i in range(len(finalResult)): # Getting all the value needed for question 1
        tempDict = {}
        
        tempDict['company_name'] = finalResult[i]['company_name']
        tempDict['company_link'] = finalResult[i]['company_link']
        tempDict['country'] = finalResult[i]['country']
        
        questionOne.append(tempDict)
        
    print('Creating company_index.json, Please Wait.')
    with open('company_index.json', 'w') as fp: # Insert the values to json file
        json.dump(questionOne, fp)
    
    print('company_index.json successfully created.')
    print('')

In [6]:
def questionTwo(finalResult):
    
    questionTwo = finalResult
    
    print('Creating company_raw.json, Please Wait.')
    with open('company_raw.json', 'w') as fp: # Insert the values to json file
        json.dump(questionTwo, fp)
    
    print('company_raw.json successfully created.')
    print('')

In [7]:
def questionThree(finalResult):
    
    questionThree = []
    
    for i in range(len(finalResult)): # preparing the values for company_parsed.json
        tempDict = {}
        tempDict['company_uid'] = finalResult[i]['company_link']
        tempDict['company_name'] = finalResult[i]['company_name']
        tempDict['country'] = finalResult[i]['country']
        tempDict['company_website'] = finalResult[i]['company_website']
        tempDict['company_address'] = finalResult[i]['company_address']
        tempDict['company_phones'] = finalResult[i]['company_phones']
        tempDict['company_faxes'] = finalResult[i]['company_fax']
        tempDict['company_industry'] = finalResult[i]['company_industry']
        
        questionThree.append(tempDict)
    
    print('Creating company_parsed.json and contact_parsed.json, Please Wait.')
    with open('company_parsed.json', 'w') as fp: # Insert the values to json file
        json.dump(questionThree, fp)
    
    print('company_parsed.json successfully created.')
    
    questionThree = []

    for i in range(len(finalResult)): # preparing the values for contact_parsed.json
        for j in finalResult[i]['company_contacts']:
            tempDict = {}
            tempDict['company_uid'] = finalResult[i]['company_link']
            tempDict['company_name'] = finalResult[i]['company_name']
            tempDict['country'] = finalResult[i]['country']
            tempDict['company_website'] = finalResult[i]['company_website']
            tempDict['contact_name'] = j['contact_name']
            tempDict['contact_job_title'] = j['job_title']
            tempDict['phone'] = j['phone']
            questionThree.append(tempDict)
    
    with open('contact_parsed.json', 'w') as fp: # Insert the values to json file
        json.dump(questionThree, fp)
    
    print('contact_parsed.json successfully created.')
    print('')

In [8]:
def main(firstPage, lastPage):
    finalResult = []

    for i in range(firstPage, (lastPage+1)): # Loops from first picked page until last picked page
        driver.get("http://www.msialogistics.com/company/list/all/" + str(i)) # Loops the page
        print('Currently Scraping Page - ' + str(i) +'.', end="\r")
        finalResult = finalResult + scrapNormal() # Appending all the list until the end

    driver.close()
    print('')
    questionOne(finalResult) # Number 1 Answer
    questionTwo(finalResult) # Number 2 Answer
    questionThree(finalResult) # Number 3 Answer
    bonusPoint(finalResult) # Bonus Point Answer
    print('Scraping Successful!')

In [9]:
driver = webdriver.Chrome()
main(1,50) # Parameter : First Page and Last Page (int)

Currently Scraping Page - 50.
Creating company_index.json, Please Wait.
company_index.json successfully created.

Creating company_raw.json, Please Wait.
company_raw.json successfully created.

Creating company_parsed.json and contact_parsed.json, Please Wait.
company_parsed.json successfully created.
contact_parsed.json successfully created.

Exporting to Excel.. Please wait.
Excel File Created. (bonusPoint.xlsx)
Scraping Successful!
